# classify

ref: https://github.com/PratyushTripathy/Landsat-Classification-Using-Convolution-Neural-Network/tree/master

https://github.com/weecology/DeepTreeAttention/blob/main/README.md (attention + pylighting fw)

## prepare dataset

### train/test/val splits + balance dataset

In [ ]:
from label_gen import get_tile_names_from_folder, load_raster_data_from_tiles, iterative_split, save_tile_names
# Generate dataset splits
resolutions = [10, 20]
directory = r"D:\Sync\research\tree_species_estimation\tree_dataset\rmf\processed"
for resolution in resolutions:
    input_folder = os.path.join(directory, f"{resolution}m", "labels", "tiles_128")
    output_folder = os.path.join(directory, f"{resolution}m", "dataset")

    # Step 1: Get tile names from the input folder
    tile_names = get_tile_names_from_folder(input_folder)

    # Step 2: Load the actual raster data from tiles
    raster_data = load_raster_data_from_tiles(input_folder, tile_names)

    # Step 3: Perform the iterative split
    train_indices, val_indices, test_indices = iterative_split(raster_data)

    # Step 4: Save the tile names into .txt files
    save_tile_names(tile_names, train_indices, val_indices, test_indices, output_folder)

In [ ]:
from osgeo import gdal
from osgeo import gdalconst
import os
from osgeo import ogr
from osgeo import osr
import fiona
from ops.ops import load_json
from tqdm.notebook import tqdm_notebook
from osgeo import gdal_array
from skimage.morphology import disk, dilation, erosion
import numpy as np

ref: 
https://github.com/klwalker-sb/burntfields_punjab

https://github.com/aime-valentin/tree_species_predictions/tree/master

https://github.com/swcoughlan/seaweed-classification

https://github.com/MitaliBhurani/Delineating-urban-areas-from-satellite-imagery/blob/master/Sentinel_imbalaced_moradabad_cv.ipynb

https://github.com/ML-MachineLearning/randomforest-GA/blob/master/random_forest.ipynb

https://github.com/AgataKisel/imagery_classification-/blob/main/random_forest.py

### Data Augmentation Strategies

## Imagery

### pylighting - UNET code

#### functions


TreeSpeciesDataModule

              ↓
 (inputs, targets, masks)  ← from DataLoader

               ↓
 Training Loop

              ↓
 MaskedMSELoss(outputs, targets, masks)
 
              ↓
 Backpropagation (only for valid pixels)


In [2]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import CSVLogger
import os
import rasterio
import torch
import torch.nn as nn
import pytorch_lightning as pl
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import numpy as np

#### loss function

In [3]:
# Custom Masked MSE Loss
class MaskedMSELoss(nn.Module):
    def __init__(self):
        super(MaskedMSELoss, self).__init__()

    def forward(self, outputs, targets, mask):
        """
        Custom MSE loss function that ignores NoData pixels.

        Args:
        - outputs: Predicted values (batch_size, num_channels, H, W)
        - targets: Ground truth values (batch_size, num_channels, H, W)
        - mask: Boolean mask indicating NoData pixels (batch_size, H, W)

        Returns:
        - loss: Mean squared error computed only for valid pixels.
        """
        # Expand mask to have the same number of channels as outputs and targets
        expanded_mask = mask.unsqueeze(1).expand_as(outputs)  # Shape: (batch_size, num_channels, H, W)
        
        # Compute squared difference between outputs and targets
        diff = (outputs - targets) ** 2

        # Zero out contributions from NoData pixels (where mask is True)
        diff = diff * (~expanded_mask)  # Keep valid pixels only

        # Sum over the channel and spatial dimensions (H, W)
        loss = diff.sum(dim=(1, 2, 3))

        # Count the number of valid pixels per batch (sum of ~mask)
        num_valid_pixels = (~expanded_mask).sum(dim=(1, 2, 3)).float()

        # Prevent division by zero (in case all pixels are NoData)
        num_valid_pixels = torch.clamp(num_valid_pixels, min=1.0)

        # Compute the mean loss per valid pixel
        loss = loss / num_valid_pixels

        # Return the mean loss over the batch
        return loss.mean()

def r2_score_torch(y_true, y_pred):
    """
    Compute the R² score in PyTorch to avoid moving tensors to CPU.
    
    Args:
    - y_true: Ground truth tensor (valid pixels, num_channels).
    - y_pred: Predicted tensor (valid pixels, num_channels).

    Returns:
    - r2: The R² score computed in PyTorch.
    """
    # Mean of the true values
    y_true_mean = torch.mean(y_true, dim=0)

    # Total sum of squares (TSS)
    total_variance = torch.sum((y_true - y_true_mean) ** 2, dim=0)

    # Residual sum of squares (RSS)
    residuals = torch.sum((y_true - y_pred) ** 2, dim=0)

    # Compute R² score for each channel and take mean
    r2 = 1 - (residuals / total_variance)
    return r2.mean()  # Mean R² across all channels

#### net

In [4]:
# PyTorch Lightning Module
class UNetLightning(pl.LightningModule):
    def __init__(self, in_channels, out_channels=9, learning_rate=1e-3):
        super(UNetLightning, self).__init__()
        
        # Define the U-Net architecture
        self.enc_conv0 = nn.Conv2d(in_channels, 64, kernel_size=3, padding=1)
        self.enc_conv1 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.enc_conv2 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.enc_conv3 = nn.Conv2d(256, 512, kernel_size=3, padding=1)

        self.dec_conv3 = nn.Conv2d(512, 256, kernel_size=3, padding=1)
        self.dec_conv2 = nn.Conv2d(256, 128, kernel_size=3, padding=1)
        self.dec_conv1 = nn.Conv2d(128, 64, kernel_size=3, padding=1)
        self.dec_conv0 = nn.Conv2d(64, out_channels, kernel_size=3, padding=1)

        self.pool = nn.MaxPool2d(2)
        self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

        self.learning_rate = learning_rate

        # Use the MaskedMSELoss
        self.criterion = MaskedMSELoss()

    def forward(self, x):
        # Encoder
        x1 = F.relu(self.enc_conv0(x))
        x2 = self.pool(x1)
        x2 = F.relu(self.enc_conv1(x2))
        x3 = self.pool(x2)
        x3 = F.relu(self.enc_conv2(x3))
        x4 = self.pool(x3)
        x4 = F.relu(self.enc_conv3(x4))

        # Decoder
        x = self.up(x4)
        x = F.relu(self.dec_conv3(x))
        x = self.up(x)
        x = F.relu(self.dec_conv2(x))
        x = self.up(x)
        x = F.relu(self.dec_conv1(x))
        x = self.dec_conv0(x)

        return x

    def training_step(self, batch, batch_idx):
        inputs, targets, masks = batch
        outputs = self(inputs)  # Forward pass
        # Expand the mask to match the number of channels in outputs and targets
        expanded_mask = masks.unsqueeze(1).expand_as(outputs)  # Shape: (batch_size, num_channels, H, W)

        # Exclude NoData pixels by applying the mask (keep only valid pixels)
        valid_outputs = outputs.masked_select(~expanded_mask).view(-1, outputs.size(1))
        valid_targets = targets.masked_select(~expanded_mask).view(-1, targets.size(1))

        # Compute the masked loss
        loss = self.criterion(outputs, targets, masks)
        # Calculate R² score for valid pixels
        r2 = r2_score_torch(valid_targets, valid_outputs)  # R² calculated in PyTorch

        # Log the training loss and R² score
        self.log('train_loss', loss, logger=True)
        self.log('train_r2', r2, logger=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        inputs, targets, masks = batch
        outputs = self(inputs)  # Forward pass
        # Expand the mask to match the number of channels in outputs and targets
        expanded_mask = masks.unsqueeze(1).expand_as(outputs)  # Shape: (batch_size, num_channels, H, W)

        # Exclude NoData pixels by applying the mask (keep only valid pixels)
        valid_outputs = outputs.masked_select(~expanded_mask).view(-1, outputs.size(1))
        valid_targets = targets.masked_select(~expanded_mask).view(-1, targets.size(1))
        
        # Compute the masked loss
        loss = self.criterion(outputs, targets, masks)
        # Calculate R² score for valid pixels
        r2 = r2_score_torch(valid_targets, valid_outputs)  # R² calculated in PyTorch

        # Log the validation loss and R² score
        self.log('val_loss', loss, logger=True)
        self.log('val_r2', r2, logger=True, prog_bar=True)
        return loss
    
    def test_step(self, batch, batch_idx):
        inputs, targets, masks = batch
        outputs = self(inputs)  # Forward pass

        expanded_mask = masks.unsqueeze(1).expand_as(outputs)
        valid_outputs = outputs.masked_select(~expanded_mask).view(-1, outputs.size(1))
        valid_targets = targets.masked_select(~expanded_mask).view(-1, targets.size(1))
        
        # Compute the masked loss
        loss = self.criterion(outputs, targets, masks)
        
        # Calculate R² score for valid pixels
        r2 = r2_score_torch(valid_targets, valid_outputs)  # R² calculated in PyTorch

        # Log the test loss and R² score
        self.log('test_loss', loss, logger=True)
        self.log('test_r2', r2, logger=True, prog_bar=True)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer


#### dataset

In [5]:
class TreeSpeciesDataset(Dataset):
    def __init__(self, tile_names, processed_dir, datasets):
        """
        Args:
            tile_names (list): List of tile filenames to load.
            processed_dir (str): Base directory containing the processed data folders.
            datasets (list): List of dataset folder names to include (e.g., ['s2/spring', 's2/summer', 'topo', 'climate']).
        """
        self.tile_names = tile_names
        self.processed_dir = processed_dir
        self.datasets = datasets  # List of dataset folder names

        # Calculate total input channels automatically
        self.total_input_channels = self.calculate_total_input_channels()

    def calculate_total_input_channels(self):
        """
        Calculate the total number of input channels by inspecting one file from each dataset.
        """
        total_channels = 0
        for dataset in self.datasets:
            example_file = os.path.join(self.processed_dir, dataset, self.tile_names[0])  # Use first tile to inspect
            with rasterio.open(example_file) as src:
                total_channels += src.count  # Add the number of bands in the dataset
        return total_channels

    def __len__(self):
        return len(self.tile_names)

    def __getitem__(self, idx):
        tile_name = self.tile_names[idx]
        input_data_list = []

        # Load data from each dataset (spring, summer, topo, climate, etc.)
        for dataset in self.datasets:
            dataset_path = os.path.join(self.processed_dir, dataset, tile_name)
            with rasterio.open(dataset_path) as src:
                input_data = src.read()  # Read the bands (num_bands, H, W)
                input_data_list.append(input_data)

        # Combine all the input data into a single input tensor
        input_data = np.concatenate(input_data_list, axis=0)  # Concatenate along the channel axis

        # Load the corresponding label (target species composition)
        label_path = os.path.join(self.processed_dir, 'labels/tiles_128', tile_name)
        with rasterio.open(label_path) as src:
            target_data = src.read()  # (num_bands, H, W)
            nodata_value_label = src.nodata  # NoData value for the labels

            # Create a NoData mask for the target data
            if nodata_value_label is not None:
                mask = np.any(target_data == nodata_value_label, axis=0)  # Collapse bands to (H, W)
            else:
                mask = np.zeros_like(target_data[0], dtype=bool)  # Assume all valid if no NoData value

        # Convert to PyTorch tensors
        input_tensor = torch.from_numpy(input_data).float()  # Shape: (total_input_channels, H, W)
        target_tensor = torch.from_numpy(target_data).float()  # Shape: (num_output_channels, H, W)
        mask_tensor = torch.from_numpy(mask).bool()  # Shape: (H, W), NoData mask

        return input_tensor, target_tensor, mask_tensor

Datamodule

In [6]:
class TreeSpeciesDataModule(pl.LightningDataModule):
    def __init__(self, tile_names, processed_dir, datasets_to_use, batch_size=4, num_workers=4):
        """
        Args:
            tile_names (list): List of tile filenames to load.
            processed_dir (str): Directory where processed data is located.
            datasets_to_use (list): List of dataset names to include (e.g., ['s2/spring', 's2/summer', 'topo']).
            batch_size (int): Batch size for DataLoader.
            num_workers (int): Number of workers for DataLoader.
        """
        super().__init__()
        self.tile_names = tile_names
        self.processed_dir = processed_dir
        self.datasets_to_use = datasets_to_use
        self.batch_size = batch_size
        self.num_workers = num_workers
        
        # Calculate total input channels based on the datasets
        temp_dataset = TreeSpeciesDataset(self.tile_names['train'], self.processed_dir, self.datasets_to_use)
        self.input_channels = temp_dataset.total_input_channels

    def setup(self, stage=None):
        """
        Sets up the dataset for train, validation, and test splits.
        """
        # Create datasets for train, validation, and test
        self.train_dataset = TreeSpeciesDataset(self.tile_names['train'], self.processed_dir, self.datasets_to_use)
        self.val_dataset = TreeSpeciesDataset(self.tile_names['val'], self.processed_dir, self.datasets_to_use)
        self.test_dataset = TreeSpeciesDataset(self.tile_names['test'], self.processed_dir, self.datasets_to_use)

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True, num_workers=self.num_workers)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, shuffle=False, num_workers=self.num_workers)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=False, num_workers=self.num_workers)


In [7]:
def load_tile_names(file_path):
    """
    Load tile names from a .txt file.

    Args:
        file_path (str): Path to the .txt file.

    Returns:
        tile_names (list): List of tile names.
    """
    with open(file_path, 'r') as f:
        tile_names = f.read().splitlines()
    return tile_names

### training

#### fall

In [26]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import CSVLogger

# User specifies which datasets to use
datasets_to_use = ['rmf_s2/fall/tiles_128']

# Tile names for train, validation, and test
tile_names = {
    'train': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m/dataset/train_tiles.txt"),
    'val': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m/dataset/val_tiles.txt"),
    'test': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m/dataset/test_tiles.txt")
}

# Initialize the DataModule
data_module = TreeSpeciesDataModule(
    tile_names=tile_names,
    processed_dir="/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m",  # Base directory where the datasets are stored
    datasets_to_use=datasets_to_use,
    batch_size=4,
    num_workers=4
)

# Use the calculated input channels from the DataModule to initialize the model
model = UNetLightning(in_channels=data_module.input_channels, learning_rate=1e-3)

# Define a checkpoint callback to save the best model
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',  # Track the validation loss
    filename='best-model-{epoch:02d}-{val_loss:.2f}',
    save_top_k=1,  # Only save the best model
    mode='min'  # We want to minimize the validation loss
)

csv_logger = CSVLogger(save_dir='logs', name="unet_s2_loss_r2")

# Create a PyTorch Lightning Trainer
trainer = Trainer(
    max_epochs=10,
    logger=[csv_logger],
    callbacks=[checkpoint_callback]
)

# Train the model
trainer.fit(model, data_module)

# Test the model after training
trainer.test(model, data_module)

# Save the best model after training
trainer.save_checkpoint("final_model.ckpt")
# Load the saved model
#model = UNetLightning.load_from_checkpoint("final_model.ckpt")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name      | Type          | Params | Mode 
-----------------------------------------------------
0  | enc_conv0 | Conv2d        | 5.2 K  | train
1  | enc_conv1 | Conv2d        | 73.9 K | train
2  | enc_conv2 | Conv2d        | 295 K  | train
3  | enc_conv3 | Conv2d        | 1.2 M  | train
4  | dec_conv3 | Conv2d        | 1.2 M  | train
5  | dec_conv2 | Conv2d        | 295 K  | train
6  | dec_conv1 | Conv2d        | 73.8 K | train
7  | dec_conv0 | Conv2d        | 5.2 K  | train
8  | pool      | MaxPool2d     | 0      | train
9  | up        | Upsample      | 0      | train
10 | criterion | MaskedMSELoss | 0      | train
-----------------------------------------------------
3.1 M     Trainable params
0         Non-trainable params
3.1 M     Total params
12.433    Total estimated model params size (MB)


Epoch 9: 100%|██████████| 186/186 [00:19<00:00,  9.33it/s, v_num=26, train_r2=0.255, val_r2=0.366]   

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 186/186 [00:19<00:00,  9.33it/s, v_num=26, train_r2=0.255, val_r2=0.366]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 40/40 [00:03<00:00, 11.61it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.03386075049638748    │
│          test_r2          │    0.3600727319717407     │
└───────────────────────────┴───────────────────────────┘

##### 10m

In [27]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import CSVLogger

# User specifies which datasets to use
datasets_to_use = ['rmf_s2/fall/tiles_128']

# Tile names for train, validation, and test
tile_names = {
    'train': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/10m/dataset/train_tiles.txt"),
    'val': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/10m/dataset/val_tiles.txt"),
    'test': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/10m/dataset/test_tiles.txt")
}

# Initialize the DataModule
data_module = TreeSpeciesDataModule(
    tile_names=tile_names,
    processed_dir="/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/10m",  # Base directory where the datasets are stored
    datasets_to_use=datasets_to_use,
    batch_size=4,
    num_workers=4
)

# Use the calculated input channels from the DataModule to initialize the model
model = UNetLightning(in_channels=data_module.input_channels, learning_rate=1e-3)

# Define a checkpoint callback to save the best model
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',  # Track the validation loss
    filename='best-model-{epoch:02d}-{val_loss:.2f}',
    save_top_k=1,  # Only save the best model
    mode='min'  # We want to minimize the validation loss
)

csv_logger = CSVLogger(save_dir='logs', name="unet_s2_loss_r2")

# Create a PyTorch Lightning Trainer
trainer = Trainer(
    max_epochs=10,
    logger=[csv_logger],
    callbacks=[checkpoint_callback]
)

# Train the model
trainer.fit(model, data_module)

# Test the model after training
trainer.test(model, data_module)

# Save the best model after training
trainer.save_checkpoint("final_model.ckpt")
# Load the saved model
#model = UNetLightning.load_from_checkpoint("final_model.ckpt")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name      | Type          | Params | Mode 
-----------------------------------------------------
0  | enc_conv0 | Conv2d        | 7.0 K  | train
1  | enc_conv1 | Conv2d        | 73.9 K | train
2  | enc_conv2 | Conv2d        | 295 K  | train
3  | enc_conv3 | Conv2d        | 1.2 M  | train
4  | dec_conv3 | Conv2d        | 1.2 M  | train
5  | dec_conv2 | Conv2d        | 295 K  | train
6  | dec_conv1 | Conv2d        | 73.8 K | train
7  | dec_conv0 | Conv2d        | 5.2 K  | train
8  | pool      | MaxPool2d     | 0      | train
9  | up        | Upsample      | 0      | train
10 | criterion | MaskedMSELoss | 0      | train
-----------------------------------------------------
3.1 M     Trainable params
0         Non-trainable params
3.1 M     Total params
12.440    Total estimated model params size (MB)


Epoch 9: 100%|██████████| 713/713 [01:18<00:00,  9.05it/s, v_num=27, train_r2=0.700, val_r2=0.447]    

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 713/713 [01:19<00:00,  9.01it/s, v_num=27, train_r2=0.700, val_r2=0.447]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 153/153 [01:10<00:00,  2.17it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │   0.029438093304634094    │
│          test_r2          │    0.4821441173553467     │
└───────────────────────────┴───────────────────────────┘

#### summer

In [6]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import CSVLogger

# Initialize the data module
data_module = TreeSpeciesDataModule(
    tiles_dir="/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m/rmf_s2/summer/tiles_128",
    labels_dir="/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m/labels/tiles_128",
    train_tile_names=load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m/dataset/train_tiles.txt"),
    val_tile_names=load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m/dataset/val_tiles.txt"),
    test_tile_names=load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m/dataset/test_tiles.txt"),
    batch_size=4,
    num_workers=4
)

# Initialize the model
model = UNetLightning(in_channels=9, out_channels=9, learning_rate=1e-3)

# Define a checkpoint callback to save the best model
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',  # Track the validation loss
    filename='best-model-{epoch:02d}-{val_loss:.2f}',
    save_top_k=1,  # Only save the best model
    mode='min'  # We want to minimize the validation loss
)

csv_logger = CSVLogger(save_dir='logs', name="unet_s2_loss_r2")

# Create a PyTorch Lightning Trainer
trainer = Trainer(
    max_epochs=10,
    logger=[csv_logger],
    callbacks=[checkpoint_callback]
)

# Train the model
trainer.fit(model, data_module)

# Test the model after training
trainer.test(model, data_module)

# Save the best model after training
trainer.save_checkpoint("final_model.ckpt")
# Load the saved model
#model = UNetLightning.load_from_checkpoint("final_model.ckpt")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A4000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name      | Type          | Params | Mode 
-----------------------------------------------------
0  | enc_conv0 | Conv2d        | 5.2 K  | train
1  | enc_conv1 | Conv2d        | 73.9 K | train
2  | enc_conv2 | Conv2d        | 295 K  | train
3  | enc_conv3 | Conv2d        | 1.2 M  | train
4  | dec_conv3 | Conv2d        | 1.2 M  | train
5  | dec_conv2 | Conv2d        | 295 K  | train
6  | dec_conv1 | Conv2d        | 73.8 K | train
7  | dec_conv0 | Conv2d        | 5.2 K  | trai

Epoch 9: 100%|██████████| 186/186 [00:18<00:00, 10.13it/s, v_num=1, train_r2=-0.273, val_r2=0.373]  

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 186/186 [00:18<00:00,  9.85it/s, v_num=1, train_r2=-0.273, val_r2=0.373]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



Testing DataLoader 0: 100%|██████████| 40/40 [00:21<00:00,  1.83it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.03334387391805649    │
│          test_r2          │    0.36800849437713623    │
└───────────────────────────┴───────────────────────────┘

##### 10m

In [42]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import CSVLogger

# User specifies which datasets to use
datasets_to_use = ['rmf_s2/summer/tiles_128']

# Tile names for train, validation, and test
tile_names = {
    'train': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/10m/dataset/train_tiles.txt"),
    'val': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/10m/dataset/val_tiles.txt"),
    'test': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/10m/dataset/test_tiles.txt")
}

# Initialize the DataModule
data_module = TreeSpeciesDataModule(
    tile_names=tile_names,
    processed_dir="/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/10m",  # Base directory where the datasets are stored
    datasets_to_use=datasets_to_use,
    batch_size=4,
    num_workers=4
)

# Use the calculated input channels from the DataModule to initialize the model
model = UNetLightning(in_channels=data_module.input_channels, learning_rate=1e-3)

# Define a checkpoint callback to save the best model
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',  # Track the validation loss
    filename='best-model-{epoch:02d}-{val_loss:.2f}',
    save_top_k=1,  # Only save the best model
    mode='min'  # We want to minimize the validation loss
)

csv_logger = CSVLogger(save_dir='logs', name="unet_s2_loss_r2")

# Create a PyTorch Lightning Trainer
trainer = Trainer(
    max_epochs=10,
    logger=[csv_logger],
    callbacks=[checkpoint_callback]
)

# Train the model
trainer.fit(model, data_module)

# Test the model after training
trainer.test(model, data_module)

# Save the best model after training
trainer.save_checkpoint("final_model.ckpt")
# Load the saved model
#model = UNetLightning.load_from_checkpoint("final_model.ckpt")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name      | Type          | Params | Mode 
-----------------------------------------------------
0  | enc_conv0 | Conv2d        | 7.0 K  | train
1  | enc_conv1 | Conv2d        | 73.9 K | train
2  | enc_conv2 | Conv2d        | 295 K  | train
3  | enc_conv3 | Conv2d        | 1.2 M  | train
4  | dec_conv3 | Conv2d        | 1.2 M  | train
5  | dec_conv2 | Conv2d        | 295 K  | train
6  | dec_conv1 | Conv2d        | 73.8 K | train
7  | dec_conv0 | Conv2d        | 5.2 K  | train
8  | pool      | MaxPool2d     | 0      | train
9  | up        | Upsample      | 0      | train
10 | criterion | MaskedMSELoss | 0      | train
-----------------------------------------------------
3.1 M     Trainable params
0         Non-trainable params
3.1 M     Total params
12.440    Total estimated model params size (MB)


Epoch 9: 100%|██████████| 713/713 [01:19<00:00,  8.92it/s, v_num=41, train_r2=0.503, val_r2=0.416]    

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 713/713 [01:19<00:00,  8.92it/s, v_num=41, train_r2=0.503, val_r2=0.416]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 153/153 [01:09<00:00,  2.21it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │   0.031047677621245384    │
│          test_r2          │     0.442976176738739     │
└───────────────────────────┴───────────────────────────┘

#### spring

In [7]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import CSVLogger

# Initialize the data module
data_module = TreeSpeciesDataModule(
    tiles_dir="/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m/rmf_s2/spring/tiles_128",
    labels_dir="/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m/labels/tiles_128",
    train_tile_names=load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m/dataset/train_tiles.txt"),
    val_tile_names=load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m/dataset/val_tiles.txt"),
    test_tile_names=load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m/dataset/test_tiles.txt"),
    batch_size=4,
    num_workers=4
)

# Initialize the model
model = UNetLightning(in_channels=9, out_channels=9, learning_rate=1e-3)

# Define a checkpoint callback to save the best model
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',  # Track the validation loss
    filename='best-model-{epoch:02d}-{val_loss:.2f}',
    save_top_k=1,  # Only save the best model
    mode='min'  # We want to minimize the validation loss
)

csv_logger = CSVLogger(save_dir='logs', name="unet_s2_loss_r2")

# Create a PyTorch Lightning Trainer
trainer = Trainer(
    max_epochs=10,
    logger=[csv_logger],
    callbacks=[checkpoint_callback]
)

# Train the model
trainer.fit(model, data_module)

# Test the model after training
trainer.test(model, data_module)

# Save the best model after training
trainer.save_checkpoint("final_model.ckpt")
# Load the saved model
#model = UNetLightning.load_from_checkpoint("final_model.ckpt")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name      | Type          | Params | Mode 
-----------------------------------------------------
0  | enc_conv0 | Conv2d        | 5.2 K  | train
1  | enc_conv1 | Conv2d        | 73.9 K | train
2  | enc_conv2 | Conv2d        | 295 K  | train
3  | enc_conv3 | Conv2d        | 1.2 M  | train
4  | dec_conv3 | Conv2d        | 1.2 M  | train
5  | dec_conv2 | Conv2d        | 295 K  | train
6  | dec_conv1 | Conv2d        | 73.8 K | train
7  | dec_conv0 | Conv2d        | 5.2 K  | train
8  | pool      | MaxPool2d     | 0      | train
9  | up        | Upsample      | 0      | train
10 | criterion | MaskedMSELoss | 0      | train
-----------------------------------------------------
3.1 M     Trainable params
0         Non-trainable params
3.1 M     Total params
12.433    Total estimated model params size (MB)


Epoch 9: 100%|██████████| 186/186 [00:18<00:00, 10.30it/s, v_num=2, train_r2=0.701, val_r2=0.365]    

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 186/186 [00:18<00:00, 10.30it/s, v_num=2, train_r2=0.701, val_r2=0.365]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 40/40 [00:10<00:00,  3.83it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.03390524163842201    │
│          test_r2          │    0.3613077998161316     │
└───────────────────────────┴───────────────────────────┘

###### 10m

In [ ]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import CSVLogger

# User specifies which datasets to use
datasets_to_use = ['rmf_s2/spring/tiles_128']

# Tile names for train, validation, and test
tile_names = {
    'train': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/10m/dataset/train_tiles.txt"),
    'val': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/10m/dataset/val_tiles.txt"),
    'test': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/10m/dataset/test_tiles.txt")
}

# Initialize the DataModule
data_module = TreeSpeciesDataModule(
    tile_names=tile_names,
    processed_dir="/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/10m",  # Base directory where the datasets are stored
    datasets_to_use=datasets_to_use,
    batch_size=4,
    num_workers=4
)

# Use the calculated input channels from the DataModule to initialize the model
model = UNetLightning(in_channels=data_module.input_channels, learning_rate=1e-3)

# Define a checkpoint callback to save the best model
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',  # Track the validation loss
    filename='best-model-{epoch:02d}-{val_loss:.2f}',
    save_top_k=1,  # Only save the best model
    mode='min'  # We want to minimize the validation loss
)

csv_logger = CSVLogger(save_dir='logs', name="unet_s2_loss_r2")

# Create a PyTorch Lightning Trainer
trainer = Trainer(
    max_epochs=10,
    logger=[csv_logger],
    callbacks=[checkpoint_callback]
)

# Train the model
trainer.fit(model, data_module)

# Test the model after training
trainer.test(model, data_module)

# Save the best model after training
trainer.save_checkpoint("final_model.ckpt")
# Load the saved model
#model = UNetLightning.load_from_checkpoint("final_model.ckpt")

#### winter

In [8]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import CSVLogger

# Initialize the data module
data_module = TreeSpeciesDataModule(
    tiles_dir="/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m/rmf_s2/winter/tiles_128",
    labels_dir="/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m/labels/tiles_128",
    train_tile_names=load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m/dataset/train_tiles.txt"),
    val_tile_names=load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m/dataset/val_tiles.txt"),
    test_tile_names=load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m/dataset/test_tiles.txt"),
    batch_size=4,
    num_workers=4
)

# Initialize the model
model = UNetLightning(in_channels=9, out_channels=9, learning_rate=1e-3)

# Define a checkpoint callback to save the best model
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',  # Track the validation loss
    filename='best-model-{epoch:02d}-{val_loss:.2f}',
    save_top_k=1,  # Only save the best model
    mode='min'  # We want to minimize the validation loss
)

csv_logger = CSVLogger(save_dir='logs', name="unet_s2_loss_r2")

# Create a PyTorch Lightning Trainer
trainer = Trainer(
    max_epochs=10,
    logger=[csv_logger],
    callbacks=[checkpoint_callback]
)

# Train the model
trainer.fit(model, data_module)

# Test the model after training
trainer.test(model, data_module)

# Save the best model after training
trainer.save_checkpoint("final_model.ckpt")
# Load the saved model
#model = UNetLightning.load_from_checkpoint("final_model.ckpt")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name      | Type          | Params | Mode 
-----------------------------------------------------
0  | enc_conv0 | Conv2d        | 5.2 K  | train
1  | enc_conv1 | Conv2d        | 73.9 K | train
2  | enc_conv2 | Conv2d        | 295 K  | train
3  | enc_conv3 | Conv2d        | 1.2 M  | train
4  | dec_conv3 | Conv2d        | 1.2 M  | train
5  | dec_conv2 | Conv2d        | 295 K  | train
6  | dec_conv1 | Conv2d        | 73.8 K | train
7  | dec_conv0 | Conv2d        | 5.2 K  | train
8  | pool      | MaxPool2d     | 0      | train
9  | up        | Upsample      | 0      | train
10 | criterion | MaskedMSELoss | 0      | train
-----------------------------------------------------
3.1 M     Trainable params
0         Non-trainable params
3.1 M     Total params
12.433    Total estimated model params size (MB)


Epoch 9: 100%|██████████| 186/186 [00:18<00:00, 10.30it/s, v_num=3, train_r2=0.252, val_r2=0.366]   

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 186/186 [00:18<00:00, 10.08it/s, v_num=3, train_r2=0.252, val_r2=0.366]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



Testing DataLoader 0: 100%|██████████| 40/40 [00:04<00:00,  8.24it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.03373197466135025    │
│          test_r2          │    0.3628885746002197     │
└───────────────────────────┴───────────────────────────┘

###### 10m

In [43]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import CSVLogger

# User specifies which datasets to use
datasets_to_use = ['rmf_s2/winter/tiles_128']

# Tile names for train, validation, and test
tile_names = {
    'train': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/10m/dataset/train_tiles.txt"),
    'val': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/10m/dataset/val_tiles.txt"),
    'test': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/10m/dataset/test_tiles.txt")
}

# Initialize the DataModule
data_module = TreeSpeciesDataModule(
    tile_names=tile_names,
    processed_dir="/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/10m",  # Base directory where the datasets are stored
    datasets_to_use=datasets_to_use,
    batch_size=4,
    num_workers=4
)

# Use the calculated input channels from the DataModule to initialize the model
model = UNetLightning(in_channels=data_module.input_channels, learning_rate=1e-3)

# Define a checkpoint callback to save the best model
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',  # Track the validation loss
    filename='best-model-{epoch:02d}-{val_loss:.2f}',
    save_top_k=1,  # Only save the best model
    mode='min'  # We want to minimize the validation loss
)

csv_logger = CSVLogger(save_dir='logs', name="unet_s2_loss_r2")

# Create a PyTorch Lightning Trainer
trainer = Trainer(
    max_epochs=10,
    logger=[csv_logger],
    callbacks=[checkpoint_callback]
)

# Train the model
trainer.fit(model, data_module)

# Test the model after training
trainer.test(model, data_module)

# Save the best model after training
trainer.save_checkpoint("final_model.ckpt")
# Load the saved model
#model = UNetLightning.load_from_checkpoint("final_model.ckpt")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name      | Type          | Params | Mode 
-----------------------------------------------------
0  | enc_conv0 | Conv2d        | 7.0 K  | train
1  | enc_conv1 | Conv2d        | 73.9 K | train
2  | enc_conv2 | Conv2d        | 295 K  | train
3  | enc_conv3 | Conv2d        | 1.2 M  | train
4  | dec_conv3 | Conv2d        | 1.2 M  | train
5  | dec_conv2 | Conv2d        | 295 K  | train
6  | dec_conv1 | Conv2d        | 73.8 K | train
7  | dec_conv0 | Conv2d        | 5.2 K  | train
8  | pool      | MaxPool2d     | 0      | train
9  | up        | Upsample      | 0      | train
10 | criterion | MaskedMSELoss | 0      | train
-----------------------------------------------------
3.1 M     Trainable params
0         Non-trainable params
3.1 M     Total params
12.440    Total estimated model params size (MB)


Epoch 9: 100%|██████████| 713/713 [01:19<00:00,  8.92it/s, v_num=42, train_r2=0.571, val_r2=0.355]    

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 713/713 [01:19<00:00,  8.92it/s, v_num=42, train_r2=0.571, val_r2=0.355]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 153/153 [00:21<00:00,  7.23it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │   0.035315658897161484    │
│          test_r2          │    0.36364343762397766    │
└───────────────────────────┴───────────────────────────┘

#### combined dataset

##### 4-season

In [8]:
# User specifies which datasets to use
datasets_to_use = ['rmf_s2/spring/tiles_128','rmf_s2/summer/tiles_128','rmf_s2/fall/tiles_128','rmf_s2/winter/tiles_128']

# Tile names for train, validation, and test
tile_names = {
    'train': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m/dataset/train_tiles.txt"),
    'val': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m/dataset/val_tiles.txt"),
    'test': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m/dataset/test_tiles.txt")
}

# Initialize the DataModule
data_module = TreeSpeciesDataModule(
    tile_names=tile_names,
    processed_dir="/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m",  # Base directory where the datasets are stored
    datasets_to_use=datasets_to_use,
    batch_size=4,
    num_workers=4
)

# Use the calculated input channels from the DataModule to initialize the model
model = UNetLightning(in_channels=data_module.input_channels, learning_rate=1e-3)

# Define a checkpoint callback to save the best model
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',  # Track the validation loss
    filename='best-model-{epoch:02d}-{val_loss:.2f}',
    save_top_k=1,  # Only save the best model
    mode='min'  # We want to minimize the validation loss
)

csv_logger = CSVLogger(save_dir='logs', name="unet_s2_loss_r2")

# Create a PyTorch Lightning Trainer
trainer = Trainer(
    max_epochs=10,
    logger=[csv_logger],
    callbacks=[checkpoint_callback]
)

# Train the model
trainer.fit(model, data_module)

# Test the model after training
trainer.test(model, data_module)

# Save the best model after training
trainer.save_checkpoint("final_model.ckpt")
# Load the saved model
#model = UNetLightning.load_from_checkpoint("final_model.ckpt")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A4000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name      | Type          | Params | Mode 
-----------------------------------------------------
0  | enc_conv0 | Conv2d        | 20.8 K | train
1  | enc_conv1 | Conv2d        | 73.9 K | train
2  | enc_conv2 | Conv2d        | 295 K  | train
3  | enc_conv3 | Conv2d        | 1.2 M  | train
4  | dec_conv3 | Conv2d        | 1.2 M  | train
5  | dec_conv2 | Conv2d        | 295 K  | train
6  | dec_conv1 | Conv2d        | 73.8 K | train
7  | dec_conv0 | Conv2d        | 5.2 K  | trai

Epoch 9: 100%|██████████| 186/186 [00:37<00:00,  4.95it/s, v_num=21, train_r2=0.444, val_r2=0.376]  

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 186/186 [00:37<00:00,  4.90it/s, v_num=21, train_r2=0.444, val_r2=0.376]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 40/40 [00:06<00:00,  6.54it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │   0.033558882772922516    │
│          test_r2          │    0.3726397454738617     │
└───────────────────────────┴───────────────────────────┘

###### 10m

In [44]:
# User specifies which datasets to use
datasets_to_use = datasets_to_use = ['rmf_s2/spring/tiles_128','rmf_s2/summer/tiles_128','rmf_s2/fall/tiles_128','rmf_s2/winter/tiles_128']

# Tile names for train, validation, and test
tile_names = {
    'train': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/10m/dataset/train_tiles.txt"),
    'val': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/10m/dataset/val_tiles.txt"),
    'test': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/10m/dataset/test_tiles.txt")
}

# Initialize the DataModule
data_module = TreeSpeciesDataModule(
    tile_names=tile_names,
    processed_dir="/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/10m",  # Base directory where the datasets are stored
    datasets_to_use=datasets_to_use,
    batch_size=4,
    num_workers=4
)

# Use the calculated input channels from the DataModule to initialize the model
model = UNetLightning(in_channels=data_module.input_channels, learning_rate=1e-3)

# Define a checkpoint callback to save the best model
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',  # Track the validation loss
    filename='best-model-{epoch:02d}-{val_loss:.2f}',
    save_top_k=1,  # Only save the best model
    mode='min'  # We want to minimize the validation loss
)

csv_logger = CSVLogger(save_dir='logs', name="unet_s2_loss_r2")

# Create a PyTorch Lightning Trainer
trainer = Trainer(
    max_epochs=10,
    logger=[csv_logger],
    callbacks=[checkpoint_callback]
)

# Train the model
trainer.fit(model, data_module)

# Test the model after training
trainer.test(model, data_module)

# Save the best model after training
trainer.save_checkpoint("final_model.ckpt")
# Load the saved model
#model = UNetLightning.load_from_checkpoint("final_model.ckpt")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name      | Type          | Params | Mode 
-----------------------------------------------------
0  | enc_conv0 | Conv2d        | 27.7 K | train
1  | enc_conv1 | Conv2d        | 73.9 K | train
2  | enc_conv2 | Conv2d        | 295 K  | train
3  | enc_conv3 | Conv2d        | 1.2 M  | train
4  | dec_conv3 | Conv2d        | 1.2 M  | train
5  | dec_conv2 | Conv2d        | 295 K  | train
6  | dec_conv1 | Conv2d        | 73.8 K | train
7  | dec_conv0 | Conv2d        | 5.2 K  | train
8  | pool      | MaxPool2d     | 0      | train
9  | up        | Upsample      | 0      | train
10 | criterion | MaskedMSELoss | 0      | train
-----------------------------------------------------
3.1 M     Trainable params
0         Non-trainable params
3.1 M     Total params
12.523    Total estimated model params size (MB)


Epoch 9: 100%|██████████| 713/713 [02:57<00:00,  4.01it/s, v_num=43, train_r2=0.355, val_r2=0.442]    

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 713/713 [02:57<00:00,  4.01it/s, v_num=43, train_r2=0.355, val_r2=0.442]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 153/153 [03:14<00:00,  0.79it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │   0.030327918007969856    │
│          test_r2          │    0.4670899212360382     │
└───────────────────────────┴───────────────────────────┘

###### summer+fall

In [46]:
# User specifies which datasets to use
datasets_to_use = datasets_to_use = ['rmf_s2/summer/tiles_128','rmf_s2/fall/tiles_128']

# Tile names for train, validation, and test
tile_names = {
    'train': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/10m/dataset/train_tiles.txt"),
    'val': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/10m/dataset/val_tiles.txt"),
    'test': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/10m/dataset/test_tiles.txt")
}

# Initialize the DataModule
data_module = TreeSpeciesDataModule(
    tile_names=tile_names,
    processed_dir="/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/10m",  # Base directory where the datasets are stored
    datasets_to_use=datasets_to_use,
    batch_size=4,
    num_workers=4
)

# Use the calculated input channels from the DataModule to initialize the model
model = UNetLightning(in_channels=data_module.input_channels, learning_rate=1e-3)

# Define a checkpoint callback to save the best model
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',  # Track the validation loss
    filename='best-model-{epoch:02d}-{val_loss:.2f}',
    save_top_k=1,  # Only save the best model
    mode='min'  # We want to minimize the validation loss
)

csv_logger = CSVLogger(save_dir='logs', name="unet_s2_loss_r2")

# Create a PyTorch Lightning Trainer
trainer = Trainer(
    max_epochs=10,
    logger=[csv_logger],
    callbacks=[checkpoint_callback]
)

# Train the model
trainer.fit(model, data_module)

# Test the model after training
trainer.test(model, data_module)

# Save the best model after training
trainer.save_checkpoint("final_model.ckpt")
# Load the saved model
#model = UNetLightning.load_from_checkpoint("final_model.ckpt")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name      | Type          | Params | Mode 
-----------------------------------------------------
0  | enc_conv0 | Conv2d        | 13.9 K | train
1  | enc_conv1 | Conv2d        | 73.9 K | train
2  | enc_conv2 | Conv2d        | 295 K  | train
3  | enc_conv3 | Conv2d        | 1.2 M  | train
4  | dec_conv3 | Conv2d        | 1.2 M  | train
5  | dec_conv2 | Conv2d        | 295 K  | train
6  | dec_conv1 | Conv2d        | 73.8 K | train
7  | dec_conv0 | Conv2d        | 5.2 K  | train
8  | pool      | MaxPool2d     | 0      | train
9  | up        | Upsample      | 0      | train
10 | criterion | MaskedMSELoss | 0      | train
-----------------------------------------------------
3.1 M     Trainable params
0         Non-trainable params
3.1 M     Total params
12.468    Total estimated model params size (MB)


Epoch 9: 100%|██████████| 713/713 [01:49<00:00,  6.51it/s, v_num=45, train_r2=-0.17, val_r2=0.450]    

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 713/713 [01:49<00:00,  6.51it/s, v_num=45, train_r2=-0.17, val_r2=0.450]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 153/153 [01:29<00:00,  1.70it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.02935178577899933    │
│          test_r2          │    0.47970086336135864    │
└───────────────────────────┴───────────────────────────┘

##### 4-season + climate

In [9]:
# User specifies which datasets to use
datasets_to_use = ['rmf_s2/spring/tiles_128','rmf_s2/summer/tiles_128','rmf_s2/fall/tiles_128','rmf_s2/winter/tiles_128', 'rmf_climate/tiles_128']

# Tile names for train, validation, and test
tile_names = {
    'train': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m/dataset/train_tiles.txt"),
    'val': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m/dataset/val_tiles.txt"),
    'test': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m/dataset/test_tiles.txt")
}

# Initialize the DataModule
data_module = TreeSpeciesDataModule(
    tile_names=tile_names,
    processed_dir="/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m",  # Base directory where the datasets are stored
    datasets_to_use=datasets_to_use,
    batch_size=4,
    num_workers=4
)

# Use the calculated input channels from the DataModule to initialize the model
model = UNetLightning(in_channels=data_module.input_channels, learning_rate=1e-3)

# Define a checkpoint callback to save the best model
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',  # Track the validation loss
    filename='best-model-{epoch:02d}-{val_loss:.2f}',
    save_top_k=1,  # Only save the best model
    mode='min'  # We want to minimize the validation loss
)

csv_logger = CSVLogger(save_dir='logs', name="unet_s2_loss_r2")

# Create a PyTorch Lightning Trainer
trainer = Trainer(
    max_epochs=10,
    logger=[csv_logger],
    callbacks=[checkpoint_callback]
)

# Train the model
trainer.fit(model, data_module)

# Test the model after training
trainer.test(model, data_module)

# Save the best model after training
trainer.save_checkpoint("final_model.ckpt")
# Load the saved model
#model = UNetLightning.load_from_checkpoint("final_model.ckpt")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name      | Type          | Params | Mode 
-----------------------------------------------------
0  | enc_conv0 | Conv2d        | 41.5 K | train
1  | enc_conv1 | Conv2d        | 73.9 K | train
2  | enc_conv2 | Conv2d        | 295 K  | train
3  | enc_conv3 | Conv2d        | 1.2 M  | train
4  | dec_conv3 | Conv2d        | 1.2 M  | train
5  | dec_conv2 | Conv2d        | 295 K  | train
6  | dec_conv1 | Conv2d        | 73.8 K | train
7  | dec_conv0 | Conv2d        | 5.2 K  | train
8  | pool      | MaxPool2d     | 0      | train
9  | up        | Upsample      | 0      | train
10 | criterion | MaskedMSELoss | 0      | train
-----------------------------------------------------
3.1 M     Trainable params
0         Non-trainable params
3.1 M     Total params
12.579    Total estimated model params size (MB)


Epoch 9: 100%|██████████| 186/186 [00:43<00:00,  4.32it/s, v_num=22, train_r2=0.555, val_r2=0.363]  

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 186/186 [00:43<00:00,  4.32it/s, v_num=22, train_r2=0.555, val_r2=0.363]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 40/40 [00:06<00:00,  5.82it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.03362555801868439    │
│          test_r2          │    0.3607933819293976     │
└───────────────────────────┴───────────────────────────┘

##### 4-season + phenology

In [33]:
# User specifies which datasets to use
datasets_to_use = ['rmf_s2/spring/tiles_128','rmf_s2/summer/tiles_128','rmf_s2/fall/tiles_128','rmf_s2/winter/tiles_128', \
    'rmf_phenology/tiles_128']

# Tile names for train, validation, and test
tile_names = {
    'train': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m/dataset/train_tiles.txt"),
    'val': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m/dataset/val_tiles.txt"),
    'test': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m/dataset/test_tiles.txt")
}

# Initialize the DataModule
data_module = TreeSpeciesDataModule(
    tile_names=tile_names,
    processed_dir="/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m",  # Base directory where the datasets are stored
    datasets_to_use=datasets_to_use,
    batch_size=4,
    num_workers=4
)

# Use the calculated input channels from the DataModule to initialize the model
model = UNetLightning(in_channels=data_module.input_channels, learning_rate=1e-3)

# Define a checkpoint callback to save the best model
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',  # Track the validation loss
    filename='best-model-{epoch:02d}-{val_loss:.2f}',
    save_top_k=1,  # Only save the best model
    mode='min'  # We want to minimize the validation loss
)

csv_logger = CSVLogger(save_dir='logs', name="unet_s2_loss_r2")

# Create a PyTorch Lightning Trainer
trainer = Trainer(
    max_epochs=10,
    logger=[csv_logger],
    callbacks=[checkpoint_callback]
)

# Train the model
trainer.fit(model, data_module)

# Test the model after training
trainer.test(model, data_module)

# Save the best model after training
trainer.save_checkpoint("final_model.ckpt")
# Load the saved model
#model = UNetLightning.load_from_checkpoint("final_model.ckpt")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name      | Type          | Params | Mode 
-----------------------------------------------------
0  | enc_conv0 | Conv2d        | 21.4 K | train
1  | enc_conv1 | Conv2d        | 73.9 K | train
2  | enc_conv2 | Conv2d        | 295 K  | train
3  | enc_conv3 | Conv2d        | 1.2 M  | train
4  | dec_conv3 | Conv2d        | 1.2 M  | train
5  | dec_conv2 | Conv2d        | 295 K  | train
6  | dec_conv1 | Conv2d        | 73.8 K | train
7  | dec_conv0 | Conv2d        | 5.2 K  | train
8  | pool      | MaxPool2d     | 0      | train
9  | up        | Upsample      | 0      | train
10 | criterion | MaskedMSELoss | 0      | train
-----------------------------------------------------
3.1 M     Trainable params
0         Non-trainable params
3.1 M     Total params
12.498    Total estimated model params size (MB)


Epoch 9: 100%|██████████| 186/186 [00:41<00:00,  4.49it/s, v_num=32, train_r2=0.713, val_r2=0.362]    

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 186/186 [00:41<00:00,  4.49it/s, v_num=32, train_r2=0.713, val_r2=0.362]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 40/40 [00:06<00:00,  5.93it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.03417683392763138    │
│          test_r2          │    0.35868608951568604    │
└───────────────────────────┴───────────────────────────┘

##### 4-season + ASTEM TOPO

In [34]:
# User specifies which datasets to use
datasets_to_use = ['rmf_s2/spring/tiles_128','rmf_s2/summer/tiles_128','rmf_s2/fall/tiles_128','rmf_s2/winter/tiles_128', \
    'rmf_topo/tiles_128']

# Tile names for train, validation, and test
tile_names = {
    'train': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m/dataset/train_tiles.txt"),
    'val': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m/dataset/val_tiles.txt"),
    'test': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m/dataset/test_tiles.txt")
}

# Initialize the DataModule
data_module = TreeSpeciesDataModule(
    tile_names=tile_names,
    processed_dir="/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m",  # Base directory where the datasets are stored
    datasets_to_use=datasets_to_use,
    batch_size=4,
    num_workers=4
)

# Use the calculated input channels from the DataModule to initialize the model
model = UNetLightning(in_channels=data_module.input_channels, learning_rate=1e-3)

# Define a checkpoint callback to save the best model
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',  # Track the validation loss
    filename='best-model-{epoch:02d}-{val_loss:.2f}',
    save_top_k=1,  # Only save the best model
    mode='min'  # We want to minimize the validation loss
)

csv_logger = CSVLogger(save_dir='logs', name="unet_s2_loss_r2")

# Create a PyTorch Lightning Trainer
trainer = Trainer(
    max_epochs=10,
    logger=[csv_logger],
    callbacks=[checkpoint_callback]
)

# Train the model
trainer.fit(model, data_module)

# Test the model after training
trainer.test(model, data_module)

# Save the best model after training
trainer.save_checkpoint("final_model.ckpt")
# Load the saved model
#model = UNetLightning.load_from_checkpoint("final_model.ckpt")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name      | Type          | Params | Mode 
-----------------------------------------------------
0  | enc_conv0 | Conv2d        | 21.4 K | train
1  | enc_conv1 | Conv2d        | 73.9 K | train
2  | enc_conv2 | Conv2d        | 295 K  | train
3  | enc_conv3 | Conv2d        | 1.2 M  | train
4  | dec_conv3 | Conv2d        | 1.2 M  | train
5  | dec_conv2 | Conv2d        | 295 K  | train
6  | dec_conv1 | Conv2d        | 73.8 K | train
7  | dec_conv0 | Conv2d        | 5.2 K  | train
8  | pool      | MaxPool2d     | 0      | train
9  | up        | Upsample      | 0      | train
10 | criterion | MaskedMSELoss | 0      | train
-----------------------------------------------------
3.1 M     Trainable params
0         Non-trainable params
3.1 M     Total params
12.498    Total estimated model params size (MB)


Epoch 9: 100%|██████████| 186/186 [00:42<00:00,  4.35it/s, v_num=33, train_r2=0.435, val_r2=0.368]   

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 186/186 [00:42<00:00,  4.35it/s, v_num=33, train_r2=0.435, val_r2=0.368]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 40/40 [00:07<00:00,  5.22it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.03379865735769272    │
│          test_r2          │    0.36142870783805847    │
└───────────────────────────┴───────────────────────────┘

##### 4-season + slope

In [35]:
# User specifies which datasets to use
datasets_to_use = ['rmf_s2/spring/tiles_128','rmf_s2/summer/tiles_128','rmf_s2/fall/tiles_128','rmf_s2/winter/tiles_128', \
    'rmf_slope/tiles_128']

# Tile names for train, validation, and test
tile_names = {
    'train': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m/dataset/train_tiles.txt"),
    'val': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m/dataset/val_tiles.txt"),
    'test': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m/dataset/test_tiles.txt")
}

# Initialize the DataModule
data_module = TreeSpeciesDataModule(
    tile_names=tile_names,
    processed_dir="/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m",  # Base directory where the datasets are stored
    datasets_to_use=datasets_to_use,
    batch_size=4,
    num_workers=4
)

# Use the calculated input channels from the DataModule to initialize the model
model = UNetLightning(in_channels=data_module.input_channels, learning_rate=1e-3)

# Define a checkpoint callback to save the best model
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',  # Track the validation loss
    filename='best-model-{epoch:02d}-{val_loss:.2f}',
    save_top_k=1,  # Only save the best model
    mode='min'  # We want to minimize the validation loss
)

csv_logger = CSVLogger(save_dir='logs', name="unet_s2_loss_r2")

# Create a PyTorch Lightning Trainer
trainer = Trainer(
    max_epochs=10,
    logger=[csv_logger],
    callbacks=[checkpoint_callback]
)

# Train the model
trainer.fit(model, data_module)

# Test the model after training
trainer.test(model, data_module)

# Save the best model after training
trainer.save_checkpoint("final_model.ckpt")
# Load the saved model
#model = UNetLightning.load_from_checkpoint("final_model.ckpt")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name      | Type          | Params | Mode 
-----------------------------------------------------
0  | enc_conv0 | Conv2d        | 21.4 K | train
1  | enc_conv1 | Conv2d        | 73.9 K | train
2  | enc_conv2 | Conv2d        | 295 K  | train
3  | enc_conv3 | Conv2d        | 1.2 M  | train
4  | dec_conv3 | Conv2d        | 1.2 M  | train
5  | dec_conv2 | Conv2d        | 295 K  | train
6  | dec_conv1 | Conv2d        | 73.8 K | train
7  | dec_conv0 | Conv2d        | 5.2 K  | train
8  | pool      | MaxPool2d     | 0      | train
9  | up        | Upsample      | 0      | train
10 | criterion | MaskedMSELoss | 0      | train
-----------------------------------------------------
3.1 M     Trainable params
0         Non-trainable params
3.1 M     Total params
12.498    Total estimated model params size (MB)


Epoch 9: 100%|██████████| 186/186 [00:40<00:00,  4.56it/s, v_num=34, train_r2=0.218, val_r2=0.380]  

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 186/186 [00:40<00:00,  4.56it/s, v_num=34, train_r2=0.218, val_r2=0.380]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 40/40 [00:06<00:00,  5.88it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.03311126306653023    │
│          test_r2          │    0.37440091371536255    │
└───────────────────────────┴───────────────────────────┘

##### 4-season + aspect

In [36]:
# User specifies which datasets to use
datasets_to_use = ['rmf_s2/spring/tiles_128','rmf_s2/summer/tiles_128','rmf_s2/fall/tiles_128','rmf_s2/winter/tiles_128', \
    'rmf_aspect/tiles_128']

# Tile names for train, validation, and test
tile_names = {
    'train': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m/dataset/train_tiles.txt"),
    'val': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m/dataset/val_tiles.txt"),
    'test': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m/dataset/test_tiles.txt")
}

# Initialize the DataModule
data_module = TreeSpeciesDataModule(
    tile_names=tile_names,
    processed_dir="/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m",  # Base directory where the datasets are stored
    datasets_to_use=datasets_to_use,
    batch_size=4,
    num_workers=4
)

# Use the calculated input channels from the DataModule to initialize the model
model = UNetLightning(in_channels=data_module.input_channels, learning_rate=1e-3)

# Define a checkpoint callback to save the best model
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',  # Track the validation loss
    filename='best-model-{epoch:02d}-{val_loss:.2f}',
    save_top_k=1,  # Only save the best model
    mode='min'  # We want to minimize the validation loss
)

csv_logger = CSVLogger(save_dir='logs', name="unet_s2_loss_r2")

# Create a PyTorch Lightning Trainer
trainer = Trainer(
    max_epochs=10,
    logger=[csv_logger],
    callbacks=[checkpoint_callback]
)

# Train the model
trainer.fit(model, data_module)

# Test the model after training
trainer.test(model, data_module)

# Save the best model after training
trainer.save_checkpoint("final_model.ckpt")
# Load the saved model
#model = UNetLightning.load_from_checkpoint("final_model.ckpt")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name      | Type          | Params | Mode 
-----------------------------------------------------
0  | enc_conv0 | Conv2d        | 21.4 K | train
1  | enc_conv1 | Conv2d        | 73.9 K | train
2  | enc_conv2 | Conv2d        | 295 K  | train
3  | enc_conv3 | Conv2d        | 1.2 M  | train
4  | dec_conv3 | Conv2d        | 1.2 M  | train
5  | dec_conv2 | Conv2d        | 295 K  | train
6  | dec_conv1 | Conv2d        | 73.8 K | train
7  | dec_conv0 | Conv2d        | 5.2 K  | train
8  | pool      | MaxPool2d     | 0      | train
9  | up        | Upsample      | 0      | train
10 | criterion | MaskedMSELoss | 0      | train
-----------------------------------------------------
3.1 M     Trainable params
0         Non-trainable params
3.1 M     Total params
12.498    Total estimated model params size (MB)


Epoch 9: 100%|██████████| 186/186 [00:41<00:00,  4.44it/s, v_num=35, train_r2=0.202, val_r2=0.400]   

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 186/186 [00:42<00:00,  4.39it/s, v_num=35, train_r2=0.202, val_r2=0.400]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 40/40 [00:07<00:00,  5.60it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │   0.031800683587789536    │
│          test_r2          │    0.3972870409488678     │
└───────────────────────────┴───────────────────────────┘

##### 4-season + trasp

In [48]:
# User specifies which datasets to use
datasets_to_use = ['rmf_s2/spring/tiles_128','rmf_s2/summer/tiles_128','rmf_s2/fall/tiles_128','rmf_s2/winter/tiles_128', \
    'rmf_twi/tiles_128']

# Tile names for train, validation, and test
tile_names = {
    'train': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m/dataset/train_tiles.txt"),
    'val': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m/dataset/val_tiles.txt"),
    'test': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m/dataset/test_tiles.txt")
}

# Initialize the DataModule
data_module = TreeSpeciesDataModule(
    tile_names=tile_names,
    processed_dir="/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m",  # Base directory where the datasets are stored
    datasets_to_use=datasets_to_use,
    batch_size=4,
    num_workers=4
)

# Use the calculated input channels from the DataModule to initialize the model
model = UNetLightning(in_channels=data_module.input_channels, learning_rate=1e-3)

# Define a checkpoint callback to save the best model
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',  # Track the validation loss
    filename='best-model-{epoch:02d}-{val_loss:.2f}',
    save_top_k=1,  # Only save the best model
    mode='min'  # We want to minimize the validation loss
)

csv_logger = CSVLogger(save_dir='logs', name="unet_s2_loss_r2")

# Create a PyTorch Lightning Trainer
trainer = Trainer(
    max_epochs=10,
    logger=[csv_logger],
    callbacks=[checkpoint_callback]
)

# Train the model
trainer.fit(model, data_module)

# Test the model after training
trainer.test(model, data_module)

# Save the best model after training
trainer.save_checkpoint("final_model.ckpt")
# Load the saved model
#model = UNetLightning.load_from_checkpoint("final_model.ckpt")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name      | Type          | Params | Mode 
-----------------------------------------------------
0  | enc_conv0 | Conv2d        | 21.4 K | train
1  | enc_conv1 | Conv2d        | 73.9 K | train
2  | enc_conv2 | Conv2d        | 295 K  | train
3  | enc_conv3 | Conv2d        | 1.2 M  | train
4  | dec_conv3 | Conv2d        | 1.2 M  | train
5  | dec_conv2 | Conv2d        | 295 K  | train
6  | dec_conv1 | Conv2d        | 73.8 K | train
7  | dec_conv0 | Conv2d        | 5.2 K  | train
8  | pool      | MaxPool2d     | 0      | train
9  | up        | Upsample      | 0      | train
10 | criterion | MaskedMSELoss | 0      | train
-----------------------------------------------------
3.1 M     Trainable params
0         Non-trainable params
3.1 M     Total params
12.498    Total estimated model params size (MB)


Epoch 0:  22%|██▏       | 41/186 [00:08<00:30,  4.80it/s, v_num=47, train_r2=nan.0]

/home/yuwei-linux/code/venv/lib/python3.10/site-packages/pytorch_lightning/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 40/40 [00:06<00:00,  5.98it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │            nan            │
│          test_r2          │            nan            │
└───────────────────────────┴───────────────────────────┘

##### 4-season + slope + aspect

In [ ]:
# User specifies which datasets to use
datasets_to_use = ['rmf_s2/spring/tiles_128','rmf_s2/summer/tiles_128','rmf_s2/fall/tiles_128','rmf_s2/winter/tiles_128', \
    'rmf_slope/tiles_128', 'rmf_aspect/tiles_128']

# Tile names for train, validation, and test
tile_names = {
    'train': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m/dataset/train_tiles.txt"),
    'val': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m/dataset/val_tiles.txt"),
    'test': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m/dataset/test_tiles.txt")
}

# Initialize the DataModule
data_module = TreeSpeciesDataModule(
    tile_names=tile_names,
    processed_dir="/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m",  # Base directory where the datasets are stored
    datasets_to_use=datasets_to_use,
    batch_size=4,
    num_workers=4
)

# Use the calculated input channels from the DataModule to initialize the model
model = UNetLightning(in_channels=data_module.input_channels, learning_rate=1e-3)

# Define a checkpoint callback to save the best model
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',  # Track the validation loss
    filename='best-model-{epoch:02d}-{val_loss:.2f}',
    save_top_k=1,  # Only save the best model
    mode='min'  # We want to minimize the validation loss
)

csv_logger = CSVLogger(save_dir='logs', name="unet_s2_loss_r2")

# Create a PyTorch Lightning Trainer
trainer = Trainer(
    max_epochs=10,
    logger=[csv_logger],
    callbacks=[checkpoint_callback]
)

# Train the model
trainer.fit(model, data_module)

# Test the model after training
trainer.test(model, data_module)

# Save the best model after training
trainer.save_checkpoint("final_model.ckpt")
# Load the saved model
#model = UNetLightning.load_from_checkpoint("final_model.ckpt")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name      | Type          | Params | Mode 
-----------------------------------------------------
0  | enc_conv0 | Conv2d        | 22.0 K | train
1  | enc_conv1 | Conv2d        | 73.9 K | train
2  | enc_conv2 | Conv2d        | 295 K  | train
3  | enc_conv3 | Conv2d        | 1.2 M  | train
4  | dec_conv3 | Conv2d        | 1.2 M  | train
5  | dec_conv2 | Conv2d        | 295 K  | train
6  | dec_conv1 | Conv2d        | 73.8 K | train
7  | dec_conv0 | Conv2d        | 5.2 K  | train
8  | pool      | MaxPool2d     | 0      | train
9  | up        | Upsample      | 0      | train
10 | criterion | MaskedMSELoss | 0      | train
-----------------------------------------------------
3.1 M     Trainable params
0         Non-trainable params
3.1 M     Total params
12.500    Total estimated model params size (MB)


Epoch 9: 100%|██████████| 186/186 [00:43<00:00,  4.23it/s, v_num=37, train_r2=0.457, val_r2=0.371]   

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 186/186 [00:44<00:00,  4.20it/s, v_num=37, train_r2=0.457, val_r2=0.371]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 40/40 [00:07<00:00,  5.62it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.03380639851093292    │
│          test_r2          │    0.3623181879520416     │
└───────────────────────────┴───────────────────────────┘

##### 4-season + slope + aspect + topo

In [41]:
# User specifies which datasets to use
datasets_to_use = ['rmf_s2/spring/tiles_128','rmf_s2/summer/tiles_128','rmf_s2/fall/tiles_128', 'rmf_s2/winter/tiles_128', \
    'rmf_topo/tiles_128', 'rmf_slope/tiles_128', 'rmf_aspect/tiles_128']

# Tile names for train, validation, and test
tile_names = {
    'train': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m/dataset/train_tiles.txt"),
    'val': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m/dataset/val_tiles.txt"),
    'test': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m/dataset/test_tiles.txt")
}

# Initialize the DataModule
data_module = TreeSpeciesDataModule(
    tile_names=tile_names,
    processed_dir="/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m",  # Base directory where the datasets are stored
    datasets_to_use=datasets_to_use,
    batch_size=4,
    num_workers=4
)

# Use the calculated input channels from the DataModule to initialize the model
model = UNetLightning(in_channels=data_module.input_channels, learning_rate=1e-3)

# Define a checkpoint callback to save the best model
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',  # Track the validation loss
    filename='best-model-{epoch:02d}-{val_loss:.2f}',
    save_top_k=1,  # Only save the best model
    mode='min'  # We want to minimize the validation loss
)

csv_logger = CSVLogger(save_dir='logs', name="unet_s2_loss_r2")

# Create a PyTorch Lightning Trainer
trainer = Trainer(
    max_epochs=10,
    logger=[csv_logger],
    callbacks=[checkpoint_callback]
)

# Train the model
trainer.fit(model, data_module)

# Test the model after training
trainer.test(model, data_module)

# Save the best model after training
trainer.save_checkpoint("final_model.ckpt")
# Load the saved model
#model = UNetLightning.load_from_checkpoint("final_model.ckpt")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name      | Type          | Params | Mode 
-----------------------------------------------------
0  | enc_conv0 | Conv2d        | 22.5 K | train
1  | enc_conv1 | Conv2d        | 73.9 K | train
2  | enc_conv2 | Conv2d        | 295 K  | train
3  | enc_conv3 | Conv2d        | 1.2 M  | train
4  | dec_conv3 | Conv2d        | 1.2 M  | train
5  | dec_conv2 | Conv2d        | 295 K  | train
6  | dec_conv1 | Conv2d        | 73.8 K | train
7  | dec_conv0 | Conv2d        | 5.2 K  | train
8  | pool      | MaxPool2d     | 0      | train
9  | up        | Upsample      | 0      | train
10 | criterion | MaskedMSELoss | 0      | train
-----------------------------------------------------
3.1 M     Trainable params
0         Non-trainable params
3.1 M     Total params
12.503    Total estimated model params size (MB)


Epoch 9: 100%|██████████| 186/186 [00:49<00:00,  3.79it/s, v_num=40, train_r2=0.741, val_r2=0.367]   

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 186/186 [00:49<00:00,  3.79it/s, v_num=40, train_r2=0.741, val_r2=0.367]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 40/40 [00:28<00:00,  1.41it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │   0.034353647381067276    │
│          test_r2          │    0.35867494344711304    │
└───────────────────────────┴───────────────────────────┘

##### 4-season + dem

In [37]:
# User specifies which datasets to use
datasets_to_use = ['rmf_s2/spring/tiles_128','rmf_s2/summer/tiles_128','rmf_s2/fall/tiles_128','rmf_s2/winter/tiles_128', \
    'rmf_dem/tiles_128']

# Tile names for train, validation, and test
tile_names = {
    'train': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m/dataset/train_tiles.txt"),
    'val': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m/dataset/val_tiles.txt"),
    'test': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m/dataset/test_tiles.txt")
}

# Initialize the DataModule
data_module = TreeSpeciesDataModule(
    tile_names=tile_names,
    processed_dir="/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m",  # Base directory where the datasets are stored
    datasets_to_use=datasets_to_use,
    batch_size=4,
    num_workers=4
)

# Use the calculated input channels from the DataModule to initialize the model
model = UNetLightning(in_channels=data_module.input_channels, learning_rate=1e-3)

# Define a checkpoint callback to save the best model
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',  # Track the validation loss
    filename='best-model-{epoch:02d}-{val_loss:.2f}',
    save_top_k=1,  # Only save the best model
    mode='min'  # We want to minimize the validation loss
)

csv_logger = CSVLogger(save_dir='logs', name="unet_s2_loss_r2")

# Create a PyTorch Lightning Trainer
trainer = Trainer(
    max_epochs=10,
    logger=[csv_logger],
    callbacks=[checkpoint_callback]
)

# Train the model
trainer.fit(model, data_module)

# Test the model after training
trainer.test(model, data_module)

# Save the best model after training
trainer.save_checkpoint("final_model.ckpt")
# Load the saved model
#model = UNetLightning.load_from_checkpoint("final_model.ckpt")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name      | Type          | Params | Mode 
-----------------------------------------------------
0  | enc_conv0 | Conv2d        | 21.4 K | train
1  | enc_conv1 | Conv2d        | 73.9 K | train
2  | enc_conv2 | Conv2d        | 295 K  | train
3  | enc_conv3 | Conv2d        | 1.2 M  | train
4  | dec_conv3 | Conv2d        | 1.2 M  | train
5  | dec_conv2 | Conv2d        | 295 K  | train
6  | dec_conv1 | Conv2d        | 73.8 K | train
7  | dec_conv0 | Conv2d        | 5.2 K  | train
8  | pool      | MaxPool2d     | 0      | train
9  | up        | Upsample      | 0      | train
10 | criterion | MaskedMSELoss | 0      | train
-----------------------------------------------------
3.1 M     Trainable params
0         Non-trainable params
3.1 M     Total params
12.498    Total estimated model params size (MB)


Epoch 9: 100%|██████████| 186/186 [00:41<00:00,  4.45it/s, v_num=36, train_r2=0.468, val_r2=0.370]   

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 186/186 [00:42<00:00,  4.42it/s, v_num=36, train_r2=0.468, val_r2=0.370]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 40/40 [00:07<00:00,  5.63it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.03354788199067116    │
│          test_r2          │    0.37181568145751953    │
└───────────────────────────┴───────────────────────────┘

##### 4-season + climate + phenology

In [ ]:
# User specifies which datasets to use
datasets_to_use = ['rmf_s2/spring/tiles_128','rmf_s2/summer/tiles_128','rmf_s2/fall/tiles_128','rmf_s2/winter/tiles_128', \
    'rmf_climate/tiles_128', 'rmf_phenology/tiles_128']

# Tile names for train, validation, and test
tile_names = {
    'train': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m/dataset/train_tiles.txt"),
    'val': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m/dataset/val_tiles.txt"),
    'test': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m/dataset/test_tiles.txt")
}

# Initialize the DataModule
data_module = TreeSpeciesDataModule(
    tile_names=tile_names,
    processed_dir="/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m",  # Base directory where the datasets are stored
    datasets_to_use=datasets_to_use,
    batch_size=4,
    num_workers=4
)

# Use the calculated input channels from the DataModule to initialize the model
model = UNetLightning(in_channels=data_module.input_channels, learning_rate=1e-3)

# Define a checkpoint callback to save the best model
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',  # Track the validation loss
    filename='best-model-{epoch:02d}-{val_loss:.2f}',
    save_top_k=1,  # Only save the best model
    mode='min'  # We want to minimize the validation loss
)

csv_logger = CSVLogger(save_dir='logs', name="unet_s2_loss_r2")

# Create a PyTorch Lightning Trainer
trainer = Trainer(
    max_epochs=10,
    logger=[csv_logger],
    callbacks=[checkpoint_callback]
)

# Train the model
trainer.fit(model, data_module)

# Test the model after training
trainer.test(model, data_module)

# Save the best model after training
trainer.save_checkpoint("final_model.ckpt")
# Load the saved model
#model = UNetLightning.load_from_checkpoint("final_model.ckpt")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name      | Type          | Params | Mode 
-----------------------------------------------------
0  | enc_conv0 | Conv2d        | 42.1 K | train
1  | enc_conv1 | Conv2d        | 73.9 K | train
2  | enc_conv2 | Conv2d        | 295 K  | train
3  | enc_conv3 | Conv2d        | 1.2 M  | train
4  | dec_conv3 | Conv2d        | 1.2 M  | train
5  | dec_conv2 | Conv2d        | 295 K  | train
6  | dec_conv1 | Conv2d        | 73.8 K | train
7  | dec_conv0 | Conv2d        | 5.2 K  | train
8  | pool      | MaxPool2d     | 0      | train
9  | up        | Upsample      | 0      | train
10 | criterion | MaskedMSELoss | 0      | train
-----------------------------------------------------
3.1 M     Trainable params
0         Non-trainable params
3.1 M     Total params
12.581    Total estimated model params size (MB)


Epoch 9: 100%|██████████| 186/186 [00:47<00:00,  3.94it/s, v_num=23, train_r2=0.614, val_r2=0.365]  

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 186/186 [00:47<00:00,  3.91it/s, v_num=23, train_r2=0.614, val_r2=0.365]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 40/40 [00:07<00:00,  5.07it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.03402198478579521    │
│          test_r2          │    0.3584168553352356     │
└───────────────────────────┴───────────────────────────┘

4-season + climate + phenology + slope

In [30]:
# User specifies which datasets to use
datasets_to_use = ['rmf_s2/spring/tiles_128','rmf_s2/summer/tiles_128','rmf_s2/fall/tiles_128','rmf_s2/winter/tiles_128', \
    'rmf_climate/tiles_128', 'rmf_phenology/tiles_128', 'rmf_slope/tiles_128']

# Tile names for train, validation, and test
tile_names = {
    'train': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m/dataset/train_tiles.txt"),
    'val': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m/dataset/val_tiles.txt"),
    'test': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m/dataset/test_tiles.txt")
}

# Initialize the DataModule
data_module = TreeSpeciesDataModule(
    tile_names=tile_names,
    processed_dir="/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m",  # Base directory where the datasets are stored
    datasets_to_use=datasets_to_use,
    batch_size=4,
    num_workers=4
)

# Use the calculated input channels from the DataModule to initialize the model
model = UNetLightning(in_channels=data_module.input_channels, learning_rate=1e-3)

# Define a checkpoint callback to save the best model
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',  # Track the validation loss
    filename='best-model-{epoch:02d}-{val_loss:.2f}',
    save_top_k=1,  # Only save the best model
    mode='min'  # We want to minimize the validation loss
)

csv_logger = CSVLogger(save_dir='logs', name="unet_s2_loss_r2")

# Create a PyTorch Lightning Trainer
trainer = Trainer(
    max_epochs=10,
    logger=[csv_logger],
    callbacks=[checkpoint_callback]
)

# Train the model
trainer.fit(model, data_module)

# Test the model after training
trainer.test(model, data_module)

# Save the best model after training
trainer.save_checkpoint("final_model.ckpt")
# Load the saved model
#model = UNetLightning.load_from_checkpoint("final_model.ckpt")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name      | Type          | Params | Mode 
-----------------------------------------------------
0  | enc_conv0 | Conv2d        | 42.7 K | train
1  | enc_conv1 | Conv2d        | 73.9 K | train
2  | enc_conv2 | Conv2d        | 295 K  | train
3  | enc_conv3 | Conv2d        | 1.2 M  | train
4  | dec_conv3 | Conv2d        | 1.2 M  | train
5  | dec_conv2 | Conv2d        | 295 K  | train
6  | dec_conv1 | Conv2d        | 73.8 K | train
7  | dec_conv0 | Conv2d        | 5.2 K  | train
8  | pool      | MaxPool2d     | 0      | train
9  | up        | Upsample      | 0      | train
10 | criterion | MaskedMSELoss | 0      | train
-----------------------------------------------------
3.1 M     Trainable params
0         Non-trainable params
3.1 M     Total params
12.583    Total estimated model params size (MB)


Epoch 9: 100%|██████████| 186/186 [00:50<00:00,  3.68it/s, v_num=29, train_r2=0.160, val_r2=0.380]    

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 186/186 [00:50<00:00,  3.68it/s, v_num=29, train_r2=0.160, val_r2=0.380]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 40/40 [00:08<00:00,  4.67it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │   0.033166274428367615    │
│          test_r2          │    0.3777430057525635     │
└───────────────────────────┴───────────────────────────┘

##### 4-season + climate + phenology + aspect

In [ ]:
# User specifies which datasets to use
datasets_to_use = ['rmf_s2/spring/tiles_128','rmf_s2/summer/tiles_128','rmf_s2/fall/tiles_128','rmf_s2/winter/tiles_128', \
    'rmf_climate/tiles_128', 'rmf_phenology/tiles_128', 'rmf_aspect/tiles_128']

# Tile names for train, validation, and test
tile_names = {
    'train': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m/dataset/train_tiles.txt"),
    'val': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m/dataset/val_tiles.txt"),
    'test': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m/dataset/test_tiles.txt")
}

# Initialize the DataModule
data_module = TreeSpeciesDataModule(
    tile_names=tile_names,
    processed_dir="/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m",  # Base directory where the datasets are stored
    datasets_to_use=datasets_to_use,
    batch_size=4,
    num_workers=4
)

# Use the calculated input channels from the DataModule to initialize the model
model = UNetLightning(in_channels=data_module.input_channels, learning_rate=1e-3)

# Define a checkpoint callback to save the best model
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',  # Track the validation loss
    filename='best-model-{epoch:02d}-{val_loss:.2f}',
    save_top_k=1,  # Only save the best model
    mode='min'  # We want to minimize the validation loss
)

csv_logger = CSVLogger(save_dir='logs', name="unet_s2_loss_r2")

# Create a PyTorch Lightning Trainer
trainer = Trainer(
    max_epochs=10,
    logger=[csv_logger],
    callbacks=[checkpoint_callback]
)

# Train the model
trainer.fit(model, data_module)

# Test the model after training
trainer.test(model, data_module)

# Save the best model after training
trainer.save_checkpoint("final_model.ckpt")
# Load the saved model
#model = UNetLightning.load_from_checkpoint("final_model.ckpt")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name      | Type          | Params | Mode 
-----------------------------------------------------
0  | enc_conv0 | Conv2d        | 42.7 K | train
1  | enc_conv1 | Conv2d        | 73.9 K | train
2  | enc_conv2 | Conv2d        | 295 K  | train
3  | enc_conv3 | Conv2d        | 1.2 M  | train
4  | dec_conv3 | Conv2d        | 1.2 M  | train
5  | dec_conv2 | Conv2d        | 295 K  | train
6  | dec_conv1 | Conv2d        | 73.8 K | train
7  | dec_conv0 | Conv2d        | 5.2 K  | train
8  | pool      | MaxPool2d     | 0      | train
9  | up        | Upsample      | 0      | train
10 | criterion | MaskedMSELoss | 0      | train
-----------------------------------------------------
3.1 M     Trainable params
0         Non-trainable params
3.1 M     Total params
12.583    Total estimated model params size (MB)


Epoch 9: 100%|██████████| 186/186 [00:51<00:00,  3.60it/s, v_num=30, train_r2=0.424, val_r2=0.396]  

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 186/186 [00:52<00:00,  3.56it/s, v_num=30, train_r2=0.424, val_r2=0.396]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 40/40 [00:08<00:00,  4.82it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.03226141259074211    │
│          test_r2          │    0.39168646931648254    │
└───────────────────────────┴───────────────────────────┘

##### 4-season + climate + phenology + topo

In [39]:
# User specifies which datasets to use
datasets_to_use = ['rmf_s2/spring/tiles_128','rmf_s2/summer/tiles_128','rmf_s2/fall/tiles_128','rmf_s2/winter/tiles_128', \
    'rmf_climate/tiles_128', 'rmf_phenology/tiles_128', 'rmf_topo/tiles_128']

# Tile names for train, validation, and test
tile_names = {
    'train': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m/dataset/train_tiles.txt"),
    'val': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m/dataset/val_tiles.txt"),
    'test': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m/dataset/test_tiles.txt")
}

# Initialize the DataModule
data_module = TreeSpeciesDataModule(
    tile_names=tile_names,
    processed_dir="/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m",  # Base directory where the datasets are stored
    datasets_to_use=datasets_to_use,
    batch_size=4,
    num_workers=4
)

# Use the calculated input channels from the DataModule to initialize the model
model = UNetLightning(in_channels=data_module.input_channels, learning_rate=1e-3)

# Define a checkpoint callback to save the best model
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',  # Track the validation loss
    filename='best-model-{epoch:02d}-{val_loss:.2f}',
    save_top_k=1,  # Only save the best model
    mode='min'  # We want to minimize the validation loss
)

csv_logger = CSVLogger(save_dir='logs', name="unet_s2_loss_r2")

# Create a PyTorch Lightning Trainer
trainer = Trainer(
    max_epochs=10,
    logger=[csv_logger],
    callbacks=[checkpoint_callback]
)

# Train the model
trainer.fit(model, data_module)

# Test the model after training
trainer.test(model, data_module)

# Save the best model after training
trainer.save_checkpoint("final_model.ckpt")
# Load the saved model
#model = UNetLightning.load_from_checkpoint("final_model.ckpt")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name      | Type          | Params | Mode 
-----------------------------------------------------
0  | enc_conv0 | Conv2d        | 42.7 K | train
1  | enc_conv1 | Conv2d        | 73.9 K | train
2  | enc_conv2 | Conv2d        | 295 K  | train
3  | enc_conv3 | Conv2d        | 1.2 M  | train
4  | dec_conv3 | Conv2d        | 1.2 M  | train
5  | dec_conv2 | Conv2d        | 295 K  | train
6  | dec_conv1 | Conv2d        | 73.8 K | train
7  | dec_conv0 | Conv2d        | 5.2 K  | train
8  | pool      | MaxPool2d     | 0      | train
9  | up        | Upsample      | 0      | train
10 | criterion | MaskedMSELoss | 0      | train
-----------------------------------------------------
3.1 M     Trainable params
0         Non-trainable params
3.1 M     Total params
12.583    Total estimated model params size (MB)


Epoch 9: 100%|██████████| 186/186 [00:49<00:00,  3.73it/s, v_num=38, train_r2=0.478, val_r2=0.387]   

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 186/186 [00:49<00:00,  3.73it/s, v_num=38, train_r2=0.478, val_r2=0.387]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 40/40 [00:10<00:00,  3.98it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.03283621370792389    │
│          test_r2          │    0.3848057985305786     │
└───────────────────────────┴───────────────────────────┘

##### all - dem

In [29]:
# User specifies which datasets to use
datasets_to_use = ['rmf_s2/spring/tiles_128','rmf_s2/summer/tiles_128','rmf_s2/fall/tiles_128', 'rmf_s2/winter/tiles_128', \
    'rmf_climate/tiles_128', 'rmf_phenology/tiles_128', \
    'rmf_topo/tiles_128', 'rmf_slope/tiles_128', 'rmf_aspect/tiles_128']

# Tile names for train, validation, and test
tile_names = {
    'train': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m/dataset/train_tiles.txt"),
    'val': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m/dataset/val_tiles.txt"),
    'test': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m/dataset/test_tiles.txt")
}

# Initialize the DataModule
data_module = TreeSpeciesDataModule(
    tile_names=tile_names,
    processed_dir="/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m",  # Base directory where the datasets are stored
    datasets_to_use=datasets_to_use,
    batch_size=4,
    num_workers=4
)

# Use the calculated input channels from the DataModule to initialize the model
model = UNetLightning(in_channels=data_module.input_channels, learning_rate=1e-3)

# Define a checkpoint callback to save the best model
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',  # Track the validation loss
    filename='best-model-{epoch:02d}-{val_loss:.2f}',
    save_top_k=1,  # Only save the best model
    mode='min'  # We want to minimize the validation loss
)

csv_logger = CSVLogger(save_dir='logs', name="unet_s2_loss_r2")

# Create a PyTorch Lightning Trainer
trainer = Trainer(
    max_epochs=10,
    logger=[csv_logger],
    callbacks=[checkpoint_callback]
)

# Train the model
trainer.fit(model, data_module)

# Test the model after training
trainer.test(model, data_module)

# Save the best model after training
trainer.save_checkpoint("final_model.ckpt")
# Load the saved model
#model = UNetLightning.load_from_checkpoint("final_model.ckpt")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name      | Type          | Params | Mode 
-----------------------------------------------------
0  | enc_conv0 | Conv2d        | 43.8 K | train
1  | enc_conv1 | Conv2d        | 73.9 K | train
2  | enc_conv2 | Conv2d        | 295 K  | train
3  | enc_conv3 | Conv2d        | 1.2 M  | train
4  | dec_conv3 | Conv2d        | 1.2 M  | train
5  | dec_conv2 | Conv2d        | 295 K  | train
6  | dec_conv1 | Conv2d        | 73.8 K | train
7  | dec_conv0 | Conv2d        | 5.2 K  | train
8  | pool      | MaxPool2d     | 0      | train
9  | up        | Upsample      | 0      | train
10 | criterion | MaskedMSELoss | 0      | train
-----------------------------------------------------
3.1 M     Trainable params
0         Non-trainable params
3.1 M     Total params
12.588    Total estimated model params size (MB)


Epoch 9: 100%|██████████| 186/186 [00:55<00:00,  3.33it/s, v_num=28, train_r2=0.176, val_r2=0.369]   

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 186/186 [00:55<00:00,  3.33it/s, v_num=28, train_r2=0.176, val_r2=0.369]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 40/40 [00:34<00:00,  1.16it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.03395329415798187    │
│          test_r2          │    0.3579249083995819     │
└───────────────────────────┴───────────────────────────┘

##### all

In [40]:
# User specifies which datasets to use
datasets_to_use = ['rmf_s2/spring/tiles_128','rmf_s2/summer/tiles_128','rmf_s2/fall/tiles_128', 'rmf_s2/winter/tiles_128', \
    'rmf_climate/tiles_128', 'rmf_phenology/tiles_128', \
    'rmf_topo/tiles_128', 'rmf_slope/tiles_128', 'rmf_aspect/tiles_128', \
    'rmf_dem/tiles_128']

# Tile names for train, validation, and test
tile_names = {
    'train': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m/dataset/train_tiles.txt"),
    'val': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m/dataset/val_tiles.txt"),
    'test': load_tile_names("/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m/dataset/test_tiles.txt")
}

# Initialize the DataModule
data_module = TreeSpeciesDataModule(
    tile_names=tile_names,
    processed_dir="/mnt/d/Sync/research/tree_species_estimation/tree_dataset/rmf/processed/20m",  # Base directory where the datasets are stored
    datasets_to_use=datasets_to_use,
    batch_size=4,
    num_workers=4
)

# Use the calculated input channels from the DataModule to initialize the model
model = UNetLightning(in_channels=data_module.input_channels, learning_rate=1e-3)

# Define a checkpoint callback to save the best model
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',  # Track the validation loss
    filename='best-model-{epoch:02d}-{val_loss:.2f}',
    save_top_k=1,  # Only save the best model
    mode='min'  # We want to minimize the validation loss
)

csv_logger = CSVLogger(save_dir='logs', name="unet_s2_loss_r2")

# Create a PyTorch Lightning Trainer
trainer = Trainer(
    max_epochs=10,
    logger=[csv_logger],
    callbacks=[checkpoint_callback]
)

# Train the model
trainer.fit(model, data_module)

# Test the model after training
trainer.test(model, data_module)

# Save the best model after training
trainer.save_checkpoint("final_model.ckpt")
# Load the saved model
#model = UNetLightning.load_from_checkpoint("final_model.ckpt")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name      | Type          | Params | Mode 
-----------------------------------------------------
0  | enc_conv0 | Conv2d        | 44.4 K | train
1  | enc_conv1 | Conv2d        | 73.9 K | train
2  | enc_conv2 | Conv2d        | 295 K  | train
3  | enc_conv3 | Conv2d        | 1.2 M  | train
4  | dec_conv3 | Conv2d        | 1.2 M  | train
5  | dec_conv2 | Conv2d        | 295 K  | train
6  | dec_conv1 | Conv2d        | 73.8 K | train
7  | dec_conv0 | Conv2d        | 5.2 K  | train
8  | pool      | MaxPool2d     | 0      | train
9  | up        | Upsample      | 0      | train
10 | criterion | MaskedMSELoss | 0      | train
-----------------------------------------------------
3.1 M     Trainable params
0         Non-trainable params
3.1 M     Total params
12.590    Total estimated model params size (MB)


Epoch 9: 100%|██████████| 186/186 [01:00<00:00,  3.08it/s, v_num=39, train_r2=0.581, val_r2=0.357]   

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 186/186 [01:00<00:00,  3.08it/s, v_num=39, train_r2=0.581, val_r2=0.357]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 40/40 [00:09<00:00,  4.02it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │   0.034369852393865585    │
│          test_r2          │    0.35076746344566345    │
└───────────────────────────┴───────────────────────────┘

rf

ref: https://github.com/shelleygoel/sentinel2-land-cover-classifier/tree/main

- Key Points:
  - X (Features): Sentinel imagery tiles stored in s2/tiles_128/ (each tile has 12 bands, size 128x128).
  - Y (Labels): The species composition tiles stored in labels/tiles_128/ (each tile has 9 bands, size 128x128). The target for each pixel is a 9-element vector representing species proportions.
  - Train/Validation/Test Splits: The tiles to use for training, validation, and testing are specified in train.txt, validation.txt, and test.txt.
- Step-by-Step Implementation:
  - Loading Data: We'll read all 1060 tiles from the directories for both input (X) and target (Y).
  - Random Forest: We'll use RandomForestRegressor to fit the data.
  - Training/Validation/Test Splits: These splits are defined by the .txt files.
  - Pixel-Wise Classification: The model will predict the species proportions for each pixel.

In [ ]:
import os
import numpy as np
import rasterio
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tqdm import tqdm

# Function to load tiles (X) and labels (Y)
def load_tile_data(tile_names, tiles_dir, labels_dir):
    """
    Load the imagery (X) and label (Y) data for the given tile names.

    Args:
        tile_names (list): List of tile names to load.
        tiles_dir (str): Directory containing the Sentinel imagery (X).
        labels_dir (str): Directory containing the species composition labels (Y).

    Returns:
        X (np.array): Flattened feature array (pixels x 12).
        Y (np.array): Flattened label array (pixels x 9).
    """
    X_list, Y_list = [], []
    print("loading data...")
    for tile_name in tqdm(tile_names):
        # Define paths for the input and label tiles
        input_tile_path = os.path.join(tiles_dir, tile_name)
        label_tile_path = os.path.join(labels_dir, tile_name)
        
        # Load input (12 bands) and label (9 bands) tiles
        with rasterio.open(input_tile_path) as src_x:
            X = src_x.read()  # Shape: (12, 128, 128)

        with rasterio.open(label_tile_path) as src_y:
            Y = src_y.read()  # Shape: (9, 128, 128)

        # Reshape to (num_pixels, num_bands)
        X_flat = X.reshape(X.shape[0], -1).T  # Shape: (num_pixels, 12)
        Y_flat = Y.reshape(Y.shape[0], -1).T  # Shape: (num_pixels, 9)

        # Append to lists
        X_list.append(X_flat)
        Y_list.append(Y_flat)
    
    # Concatenate all tiles into a single array
    X_all = np.vstack(X_list)
    print(f"shape of labels: {X_all.shape}")
    Y_all = np.vstack(Y_list)
    print(f"shape of labels: {Y_all.shape}")
    
    return X_all, Y_all

# Function to read the train/validation/test splits
def load_split(file_path):
    """
    Load the tile names from the train/validation/test split files.

    Args:
        file_path (str): Path to the split .txt file.

    Returns:
        tile_names (list): List of tile names in the split.
    """
    with open(file_path, 'r') as file:
        tile_names = file.read().splitlines()
    return tile_names

# Set up directories
directory = r"D:\Sync\research\tree_species_estimation\tree_dataset\rmf\processed\10m"
tiles_dir = os.path.join(directory, "rmf_s2", "summer", "tiles_128")  # Directory for X
labels_dir = os.path.join(directory, "labels", "tiles_128")  # Directory for Y

# Load train/validation/test splits
train_tile_names = load_split(os.path.join(directory, "dataset", "train_tiles.txt"))[:400]
val_tile_names = load_split(os.path.join(directory, "dataset", "val_tiles.txt"))[:100]

# Load the training data
X_train, Y_train = load_tile_data(train_tile_names, tiles_dir, labels_dir)

# Load the validation data (optional, but useful for hyperparameter tuning)
X_val, Y_val = load_tile_data(val_tile_names, tiles_dir, labels_dir)

# Initialize and train the Random Forest model
print("start training...")
rf = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf.fit(X_train, Y_train)

# Evaluate on the validation set
print("validating...")
Y_val_pred = rf.predict(X_val)
val_mse = mean_squared_error(Y_val, Y_val_pred)
print(f"Validation Mean Squared Error: {val_mse}")


loading data...


100%|██████████| 400/400 [01:40<00:00,  3.99it/s]


shape of labels: (6553600, 12)
shape of labels: (6553600, 9)
loading data...


100%|██████████| 100/100 [00:34<00:00,  2.93it/s]


shape of labels: (1638400, 12)
shape of labels: (1638400, 9)
start training...
validating...
Validation Mean Squared Error: 0.21201176008304376


In [ ]:
import joblib
# After training the model
rf = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf.fit(X_train, Y_train)

# Save the model to a file
model_filename = 'random_forest_model.joblib'
joblib.dump(rf, model_filename)

print(f"Model saved as {model_filename}")

# Load the saved model
loaded_rf = joblib.load(model_filename)

# Load the testing data (for final evaluation)
test_tile_names = load_split(os.path.join(directory, "dataset", "test_tiles.txt"))
X_test, Y_test = load_tile_data(test_tile_names, tiles_dir, labels_dir)

# Use the loaded model to make predictions
Y_test_pred = loaded_rf.predict(X_test)
test_mse = mean_squared_error(Y_test, Y_test_pred)
print(f"Test Mean Squared Error with loaded model: {test_mse}")


LUCinSA_helpers
Helper functions and notebooks to interact with data on High-Performance Computing environment, designed to be used in conjunction with processing guide for remote sensing projects on Land-Use Change in Latin America:

https://github.com/klwalker-sb/LUCinSA_helpers/tree/master

https://klwalker-sb.github.io/LUCinLA_stac/Downloading.html

## fusion with pts